In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd 
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import copy

from PIL import Image
import imageio

from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda_is_available() else "cpu")
print(device)

In [1]:
csv_dir_prefix = os.path.join(".", "data", "relabel_")

train_df = pd.read_csv(csv_dir_prefix + "train.csv")
val_df = pd.read_csv(csv_dir_prefix + "l_val.csv")
test_df = pd.read_csv(csv_dir_prefix + "_test.csv")

print("Training Data:" , train_data.shape[0])
print("Validation Data:" , val_data.shape[0])
print("Testing Data:" , test_data.shape[0])

NameError: name 'pd' is not defined

In [2]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_data(df, data_dir="./data/train"):
    x = []
    y = df["landmark_id"]

    for id in tqdm(df["id"]):
        img_path = os.path.join(data_dir, id[0], id[1], id[2], id+".jpg")
        img = Image.open(img_path)
        x.append(preprocess(image))
    
    return x, y

NameError: name 'transforms' is not defined

In [ ]:
x_train, y_train = load_data(train_df, data_dir="./data/train_discard_6")
x_val, y_val = load_data(val_df, data_dir="./data/val_discard_6")
x_test, y_test = load_data(test_df, data_dir="./data/test_discard_6")

data = {"train": x_train, "val": x_val, "test": x_test}
dataloaders = {x: torch.utils.data.DataLoader(data[x], batch_size=4, shuffle=True, num_workers=4) for x in ["train", "val", "test"]}
dataset_sizes = {x: len(data[x]) for x in ["train", "val", "test"]}

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model = EfficientNet.from_pretrained("efficientnet-b0")
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 12)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=10)